In [ ]:
# Time forecasting workflow 

This is the workflow for running a time range of simulations (like hincast or re-forecasting). See workflow.py script. It runs from rerun.py script.

In [1]:
#Python modules
import numpy as np
import datetime
import sys
import os
from shutil import copy2
import glob
import pickle
import subprocess


In [2]:
%load_ext pypath_magic

In [3]:
# add temporarily to the python the required local paths. Make sure you clean up afterwards.
# It won't run if ~/pypath_magic.pth is not empty to begin with.
%pypath -a /home/critechuser/REPOS/StormS/src/
%pypath -a /home/critechuser/REPOS/StormS/utils/

Added u'/home/critechuser/REPOS/StormS/src' to path.
Added u'/home/critechuser/REPOS/StormS/utils' to path.


In [4]:
#local modules
from meteo import wmap
from idelft3d import meteo2delft3d
import mdf

In [5]:
# clean up 
%pypath -d /home/critechuser/REPOS/StormS/src/
%pypath -d /home/critechuser/REPOS/StormS/utils/

Deleted u'/home/critechuser/REPOS/StormS/src' from path
Deleted u'/home/critechuser/REPOS/StormS/utils' from path


## Given variables

In [6]:
path = 'tmp/'

In [36]:
# check contents of folder
tfolder = glob.glob(path+'*')
tfolder

['tmp/20170212.12', 'tmp/20170213.00', 'tmp/tmp.pkl', 'tmp/20170212.00']

Now we define the run from time stamp to time stamp e.g. '20170212.12' to '201702.20.00'. 
The start time stamp of the follow up run is 12 hours after the last simulation present in the folder. 

In [37]:
# exclude pkl file from contents of folder
tfolder = [t for t in tfolder if '.pkl' not in t]
tfolder

['tmp/20170212.12', 'tmp/20170213.00', 'tmp/20170212.00']

In [42]:
tl = [t.split('/')[-1] for t in tfolder]

In [44]:
tl.sort() # we sort to get the last one
tl

['20170212.00', '20170212.12', '20170213.00']

In [11]:
rundate = datetime.datetime.strptime(tl[-1], '%Y%m%d.%H')

In [12]:
# Set time stamp of the follow up run for 12 hours after the last simulation
rundate = rundate + datetime.timedelta(hours=12) 

In [13]:
pfile = glob.glob(path+'*.pkl')[0]
pfile

'tmp/tmp.pkl'

In [14]:
with open(pfile, 'r') as f:
    dic=pickle.load(f)


In [15]:
dic

{'bname': 'test',
 'lat0': 25.0,
 'lat1': 76.1,
 'lon0': -47.0,
 'lon1': 44.10000000000001,
 'ni': 911,
 'nj': 511,
 'nt': 12}

In [16]:
bname=dic['bname']
nt=dic['nt']
ni=dic['ni']
nj=dic['nj']
lon0=dic['lon0']
lat0=dic['lat0']
lon1=dic['lon1']
lat1=dic['lat1']

In [17]:
#files to be copied
files=['config_d_hydro.xml',bname+'.mdf',bname+'.grd',bname+'.enc',bname+'.obs',bname+'.dep', bname+'.pkl','run_flow2d3d.sh']

In [18]:
# previous date for reference
previous_run=rundate-datetime.timedelta(hours=12)

In [19]:
# create the folder/run path
folder=datetime.datetime.strftime(rundate,'%Y%m%d.%H' )

rpath=path+'{}/'.format(folder)

if not os.path.exists(rpath):
    os.makedirs(rpath)

In [20]:
# copy necessary files
pfolder=datetime.datetime.strftime(previous_run,'%Y%m%d.%H' )
ppath =  path+'{}/'.format(pfolder)  # previous path

for filename in files:
      copy2(ppath+filename,rpath+filename)

In [21]:
# copy restart file from previous time stamp folder

inresfile='tri-rst.'+bname+'.'+datetime.datetime.strftime(rundate,'%Y%m%d.%H%M%M')

outresfile='restart.'+datetime.datetime.strftime(rundate,'%Y%m%d.%H%M%M')

copy2(ppath+inresfile,rpath+'tri-rst.'+outresfile)

In [22]:
#get new meteo
check=[os.path.exists(rpath+f) for f in ['u.amu','v.amv','p.amp']] # check if u,v,p are there to avoid duplication of effort
if np.any(check)==False :
    p,u,v,lat,lon = wmap(rundate,0,3*(nt+1),lon0,lon1,lat0,lat1)
    dlat=lat[1,0]-lat[0,0]
    dlon=lon[0,1]-lon[0,0]

    meteo2delft3d(p,u,v,lat0,lon0,dlat,dlon,rundate,nt,path=rpath,curvi=False)

100%|██████████| 39/39 [02:01<00:00,  2.89s/it]      | 1/39 [00:04<02:53,  4.58s/it]



meteo done


In [23]:
# modify mdf file. 

inp, order = mdf.read(rpath+bname+'.mdf') #read 

# adjust iteration date
tstart=rundate.hour*60
inp['Itdate']=datetime.datetime.strftime(rundate,'%Y-%m-%d')
inp['Tstart']=[tstart]
inp['Tstop']=[nt*60+tstart]
inp['Flmap']  = [tstart,60,nt*60+tstart] # it is assumed save map every 60 minutes
inp['Flhis']  = [tstart,inp['Dt'][0],nt*60+tstart] # it is assumed save obs point every timestep


if 'Restid' not in order : order.append('Restid')
 # adjust restart file
inp['Restid']=outresfile

# update mdf
mdf.write(inp, rpath+bname+'.mdf',selection=order)



In [32]:
# run case
# note that cwd is the folder where the executable is
ex=subprocess.Popen('./run_flow2d3d.sh', cwd=rpath, shell=True, stdout=subprocess.PIPE, bufsize=1)
for line in iter(ex.stdout.readline,b''): print line
ex.stdout.close()

KeyboardInterrupt: 